In [0]:
Clinicaltrial_2021 = "/FileStore/tables/clinicaltrial_2021.csv"
Clinicaltrial_2020 = "/FileStore/tables/clinicaltrial_2020.csv"
Clinicaltrial_2019 = "/FileStore/tables/clinicaltrial_2019.csv"
Mesh = "/FileStore/tables/mesh.csv"
Pharma = "/FileStore/tables/pharma.csv"

In [0]:
###creating a variable for reusability
study_year = '2021'

In [0]:
###Including the variable in the filepath
"clinicaltrial_" + study_year + ".csv"

Out[182]: 'clinicaltrial_2021.csv'

In [0]:
#reading the Mesh file into an RDD

MeshRDD = sc.textFile(Mesh)

In [0]:
#reading the Pharma file into an RDD

pharmaRDD = sc.textFile(Pharma)

In [0]:
clinicaltrialRDD = sc.textFile("FileStore/tables/" + "clinicaltrial_" + study_year + ".csv")

In [0]:
clinicaltrialRDD.take(5)

Out[186]: ['Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions',
 'NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||',
 'NCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|',
 'NCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|',
 'NCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|']

In [0]:
firstline = clinicaltrialRDD.first()

In [0]:
ClinicalHeaderRDD = clinicaltrialRDD.filter(lambda x : x != firstline)

In [0]:
# Retrieving the distinct count from 2021 dataset

ClinicalHeaderRDD.count()

Out[189]: 387261

In [0]:
#Retrieving all the types of studies in the dataset along with the frequency

TypeRDD = ClinicalHeaderRDD.map(lambda s: s.split("|"))\
.map(lambda s: s[5])\
.map(lambda s: (s,1))\
.reduceByKey(lambda s,y: s+y)\
.sortBy(lambda s: s[1], ascending=False)

In [0]:
TypeRDD.take(5)

Out[191]: [('Interventional', 301472),
 ('Observational', 77540),
 ('Observational [Patient Registry]', 8180),
 ('Expanded Access', 69)]

In [0]:
#Retrieving the top 5 conditions and their frequencies

ConditionsRDD = ClinicalHeaderRDD.map(lambda s: s.split("|"))\
.map(lambda s: s[7])\
.filter(lambda s: s!='')\
.flatMap(lambda s: s.split(","))\
.map(lambda s: (s,1))\
.reduceByKey(lambda s,y: s+y)\
.sortBy(lambda s: s[1], ascending=False)

In [0]:
ConditionsRDD.take(5)

Out[193]: [('Carcinoma', 13389),
 ('Diabetes Mellitus', 11080),
 ('Neoplasms', 9371),
 ('Breast Neoplasms', 8640),
 ('Syndrome', 8032)]

In [0]:
#Retrieving the top 5 most frequent roots

split_MeshRDD = MeshRDD.map(lambda s: s.split(","))

In [0]:
split_conditionsRDD = ClinicalHeaderRDD.map(lambda s: s.split("|") [7])\
.filter(lambda s: s!='')\
.flatMap(lambda s: s.split(","))\
.map(lambda s: (s,1))

In [0]:
frequentrootsRDD = split_conditionsRDD.join(split_MeshRDD)


In [0]:
toprootsRDD = frequentrootsRDD.map(lambda s: s[1])\
.map(lambda s: s[1])\
.map(lambda s: s.split(".") [0])\
.map(lambda s: (s,1))\
.reduceByKey(lambda s,y: s+y)\
.sortBy(lambda s: s[1], ascending=False)

In [0]:
toprootsRDD.take(5)

Out[198]: [('C04', 143994),
 ('C23', 136079),
 ('C01', 106674),
 ('C14', 94523),
 ('C10', 92310)]

In [0]:
#Retrieving the ten most common sponsors that are not Pharmaceutical companies


split_pharmaRDD = pharmaRDD.map(lambda s: s.replace('"','')).map(lambda s: s.split(","))\
.map(lambda s: s[1])\
.map(lambda s: (s,1))



In [0]:
sponsorRDD = ClinicalHeaderRDD.map(lambda s: s.split("|"))\
.map(lambda s: s[1])\
.map(lambda s: (s,1))\



In [0]:
nonpharmaRDD = sponsorRDD.subtract(split_pharmaRDD)

In [0]:
top_nonpharmaRDD = nonpharmaRDD.reduceByKey(lambda s,y: s+y)\
.sortBy(lambda s: s[1], ascending=False)

In [0]:
top_nonpharmaRDD.take(10)

Out[203]: [('National Cancer Institute (NCI)', 3218),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088),
 ('Massachusetts General Hospital', 1971),
 ('Cairo University', 1928),
 ('Hoffmann-La Roche', 1828)]

In [0]:
#number of completed studies in each month of the year

#splitting the ClinicalHeaderRDD by the delimiter and mapping the relevant columns
completedstudiesRDD = ClinicalHeaderRDD.map(lambda s: s.split("|"))\
.map(lambda s: (s[2],s[4]))\
.filter(lambda s: s[0]=='Completed')

In [0]:
filteredyearsRDD = completedstudiesRDD.map(lambda s: s[1])\
.filter(lambda s: study_year in s)

In [0]:
splitRDD = filteredyearsRDD.map(lambda s: s.split(' '))\
.map(lambda s: s[0])\
.map(lambda s: (s,1))\
.reduceByKey(lambda s,y: s+y)\
.sortBy(lambda s: s[1], ascending=False)

In [0]:
splitRDD.take(12)

Out[207]: [('Mar', 1227),
 ('Jan', 1131),
 ('Jun', 1094),
 ('May', 984),
 ('Apr', 967),
 ('Feb', 934),
 ('Jul', 819),
 ('Aug', 700),
 ('Sep', 528),
 ('Oct', 187)]

In [0]:
import calendar

In [0]:
p = {z : m for m,z in enumerate (calendar.month_abbr[1:],1)}
print(p)

{'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}


In [0]:
arrangedRDD = splitRDD.sortBy(keyfunc = lambda s: p.get(s[0]))

In [0]:
arrangedRDD.take(12)

Out[211]: [('Jan', 1131),
 ('Feb', 934),
 ('Mar', 1227),
 ('Apr', 967),
 ('May', 984),
 ('Jun', 1094),
 ('Jul', 819),
 ('Aug', 700),
 ('Sep', 528),
 ('Oct', 187)]